# Mail编程

## 电子邮件的历史
- 起源
    - 1969 Leonard K. 教授发给同事的 "LO"
    - 1971 美国国防部自主的阿帕网(Arpanet)的通讯机制
    - 通讯地址里用@
    - 1987年中国的第一份电子邮件
        - "Across the Great Wall we can reach every corner in the world"
- 管理程序
    - Euroda使邮件普及
    - Netscape, outlook, forxmail后来居上
    - Hotmal使用浏览器发送邮件
- 参考资料
    - [官网](https://docs.python.org/3/library/email.mime.html)

## 邮件工作流程
- MUA(MailUserAgent)邮件用户代理
- MTA(MailTransferAgent)邮件传输代理
- MDA(MailDeliveryAgent)邮件投递代理
    - 举例
    - laoshi@qq.com, 老师的是QQ信箱
    - xuesheng@sina.com, 学生的是新浪信箱
    - 流程
         1. MUA -> MTA, 邮件已经在服务器上了
         2. qq MTA -> ....... -> sina MTA, 邮件在新浪的服务器上
         3. sina MTA -> sina MDA, 此时邮件已经在你的邮箱里了
         4. sina MDA -> MUA(Foxmail/Outlook), 邮件下载到本地电脑
         
- 编写程序
    - 发送: MUA -> MTA with SMTP:SimpleMailTransferProtocal, 包含MTA -> MTA
    - 接受: MDA -> MUA with POP3 and IMAP: PostOfficeProtocal v3 and InternetMessageAccessProtocal v4
    
- 准备工作
    - 注册邮箱(以qq邮箱为例)
    - 第三方邮箱需要特殊设置, 以qq邮箱为例
        - 进入设置中心
        - 取得授权码
- python for mail
    - SMTP协议负责发送邮件
        - 使用email模块构建邮件
            - 纯文本邮件
            - 案例 v01
            
        - HTML格式邮件发送
            - 准备HTML代码作为内容
            - 把邮件的subtpye设为html
            - 案例 v02
            
        - 发送带附件的邮件
            - 可以把邮件看作是一个文本邮件和一个附件的合体
            - 一封信如果涉及多个部分, 需要使用MIMEMultipart格式构建
            - 添加一个MIMEText正文
            - 添加一个MIMEBase或者MIMEText作为附件
            - 案例 v03
            
        - 添加邮件头(摘要), 抄送等信息
            - mail属性
            - mail["From"]: 表示发送者信息, 包括姓名和邮件
            - mali["To"]: 表示接收者信息, 包括姓名和邮件地址
            - mail["Subject"]: 表示摘要或者主题信息
            - 案例 v04
            
        - 同时支持html和text格式
            - 构建一个MIMEMultipart格式邮件
            - MIMEMultipart的subtype设置成alternative格式
            - 添加HTML和text邮件 能显示HTML的优先显示 不支持的就显示text
            - 案例 v05
        
        - 使用smplib模块发送邮件
        
    - POP3协议负责接受邮件
        - 本质上是MDA到MUA的一个过程
        - 从MDA下载下来的,是一个完整的邮件结构体, 需要解析才能得到具体内容
        - 步骤:
            - 1. 用poplib下载邮件结构体原始内容
                - 准备相应的内容 (邮件地址,密码,POP3实例)
                - 身份认证
                - 一般会先得到邮件内邮件的整体列表
                - 根据相应序号, 得到某一封信的数据流
                - 利用解析函数进行解析出相应的邮件结构体
            - 2. 用email解析邮件的具体内容
            - 案例 v06

In [1]:
# 案例 v01
# 导入相应的包
import smtplib
from email.mime.text import MIMEText
# MIMEText三个主要参数
# 1. 邮件内容
# 2. MIME子类型, 在此案例我们用plain表示text类型
# 3. 邮件编码格式

msg = MIMEText("Hello, i am beijing tulingxueyuan", "plain", "utf-8")

# 发送email地址, 此地址直接使用我的qq邮箱, 密码一般需要临时输入
from_addr = "249399289@qq.com"

# 此处密码是经过申请设置后的授权码, 不是你的qq邮箱密码
from_pwd = 'ckvpjeqjbnjxbicd'

# 收件人信息
# 此处使用qq邮箱, 发给我自己
to_addr = "249399289@qq.com"

# 输入SMTP服务器地址
# 此处根据不用的邮件服务商 有不同的值
# 现在基本任何一家邮件服务商, 如果采用第三方收发邮件, 都需要开启授权选项
# 腾讯qq预想所用的smtp地址是 smtp.qq.com

smtp_stv = 'smtp.qq.com'

try:
    # 第一个参数是服务器地址, 但一定是bytes格式, 所以需要编码
    # 第二个参数是服务器的接收访问端口
    srv = smtplib.SMTP_SSL(smtp_stv.encode(), 465) # SMTP协议默认端口是25
    # 登录邮箱发送
    srv.login(from_addr, from_pwd)
    # 发送邮件 三个参数
    # 1. 发送地址
    # 2. 接收地址,必须是list格式
    # 3. 发送内容, 作为字符串发送
    srv.sendmail(from_addr, [to_addr], msg.as_string())
    srv.quit()
except Exception as e:
    print(e)

In [6]:
# 案例 v02
from email.mime.text import MIMEText

mail_content = '''
        <!DOCTYOE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>Title</title>
        </head>
        <body>
        
        <h1> 这是一封HTML格式邮件</h1>
        
        </body>
        </html>
'''

msg = MIMEText(mail_content, "html", "utf-8")

# 构建发送者地址和登录信息
from_addr = "249399289@qq.com"
from_pwd = "ckvpjeqjbnjxbicd"

# 构建邮件接受者信息
to_addr = "249399289@qq.com"

smtp_stv = "smtp.qq.com"

try:
    import smtplib
    
    srv = smtplib.SMTP_SSL(smtp_stv.encode(), 465)
    
    srv.login(from_addr, from_pwd)
    srv.sendmail(from_addr, [to_addr], msg.as_string())
    srv.quit()
except Exception as e:
    print(e)

In [10]:
# 案例 v03
from email.mime.text import MIMEText # 构建附件使用
from email.mime.multipart import MIMEBase, MIMEMultipart  # 构建基础邮件使用

# 生成
mail_mul = MIMEMultipart()

# 构建正文
mail_text = MIMEText("Hello, i am Mingming", "plain", "utf-8")

# 把构建好的邮件正文附加进邮件中
mail_mul.attach(mail_text)

# 构建附加 需要从本地读取附件
# 以rb格式打开
with open("02.html", "rb") as f:
    s = f.read()
    
    # 设置附件的MIME和文件名
    m = MIMEText(s, "base64", "utf-8")
    m["Content-Type"] = "application/octet-stream"
    
    # 1.attachment 分号为英文状态
    # 2. fiename 后面需要用引号包裹, 注意与外面引号错开
    m["Content-Disposition"] = "attachment; filename='02.html'"
    
    # 添加到MIMEMultipart
    mail_mul.attach(m)
    
# 构建发送者地址和登录信息
from_addr = "249399289@qq.com"
from_pwd = "ckvpjeqjbnjxbicd"

# 构建邮件接受者信息
to_addr = "249399289@qq.com"

smtp_stv = "smtp.qq.com"

try:
    import smtplib
    
    srv = smtplib.SMTP_SSL(smtp_stv.encode(), 465)
    
    srv.login(from_addr, from_pwd)
    srv.sendmail(from_addr, [to_addr], mail_mul.as_string())
    srv.quit()
except Exception as e:
    print(e)

In [17]:
# 案例 v04
from email.mime.text import MIMEText
from email.header import Header

msg = MIMEText("Hello wold", "plain", "utf-8")

header_from = Header("我自己 <249399289@qq.com>","utf-8")
msg['From'] = header_from

header_to = Header("<249399289@qq.com>","utf-8")
msg['To'] = header_to

header_sub = Header("使用pthon发邮件","utf-8")
msg["Subject"] = header_sub

# 构建发送者地址和登录信息
from_addr = "249399289@qq.com"
from_pwd = "ckvpjeqjbnjxbicd"

# 构建邮件接受者信息
to_addr = "249399289@qq.com"

smtp_stv = "smtp.qq.com"

try:
    import smtplib
    
    srv = smtplib.SMTP_SSL(smtp_stv.encode(), 465)
    
    srv.login(from_addr, from_pwd)
    srv.sendmail(from_addr, [to_addr], msg.as_string())
    srv.quit()
except Exception as e:
    print(e)

In [18]:
# 案例 v05
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# 构建一个MIMEMultipart邮件
msg = MIMEMultipart("alternative")

# 构建一个HTML邮件内容
html_content = '''
        <!DOCTYOE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>Title</title>
        </head>
        <body>
        
        <h1> 这是一封HTML格式邮件</h1>
        
        </body>
        </html>
'''

# 添加
msg_html = MIMEText(html_content, "html", "utf-8")
msg.attach(msg_html)

msg_text = MIMEText('just text content', 'plain', 'utf-8')
msg.attach(msg_text)

# 构建发送者地址和登录信息
from_addr = "249399289@qq.com"
from_pwd = "ckvpjeqjbnjxbicd"

# 构建邮件接受者信息
to_addr = "249399289@qq.com"

smtp_stv = "smtp.qq.com"

try:
    import smtplib
    
    srv = smtplib.SMTP_SSL(smtp_stv.encode(), 465)
    
    srv.login(from_addr, from_pwd)
    srv.sendmail(from_addr, [to_addr], msg.as_string())
    srv.quit()
except Exception as e:
    print(e)

In [ ]:
# 案例 v06